In [ ]:
import nibabel
import matplotlib.pyplot as plt
import numpy as np
import pathlib

In [2]:


ct_scan_folder = pathlib.Path(r"D:\Studia\sem7\inzynierka\data\ct_scan")
# lung_mask_folder = pathlib.Path(r"D:\Studia\sem7\inzynierka\data\lung_mask")
image_file = "coronacases_org_001.nii"
# mask_file = "coronacases_001.nii"

# Load image files with nibabel 
CT = nibabel.load(ct_scan_folder / image_file)

CT_array = CT.get_fdata()

In [24]:
image3D  = sitk.ReadImage(str(ct_scan_folder / image_file)) 
castFilter = sitk.CastImageFilter()
castFilter.SetOutputPixelType(sitk.sitkInt16)
# Convert floating type image (imgSmooth) to int type (imgFiltered)
imgFiltered = castFilter.Execute(image3D)

In [36]:
# https://github.com/SimpleITK/SimpleITK/issues/378

nii_data_path = r"D:\Studia\sem7\inzynierka\data\eksperyment"
dicom_output_path = r"D:\Studia\sem7\inzynierka\data\eksperyment"
patients = os.listdir(nii_data_path)
patients.sort()

for s in patients:
    NiftyImg = sitk.ReadImage(nii_data_path + "\\" + s)
    Img_array = sitk.GetArrayViewFromImage(NiftyImg)
    new_img = sitk.GetImageFromArray(Img_array)
    new_img.SetSpacing([2.5,3.5,4.5])
    Slic_Num = len(Img)
    print(s, Slic_Num)
    for slic in range(Slic_Num):
        DicomFileName = dicom_output_path + s.split('.nii')[0] + "-" + str(slic) + ".dcm"
        writer.SetFileName(DicomFileName)        
#         sitk.WriteImage(Img[slic,:,:], DicomFileName)
        writer.Execute(new_img[:,:,slic])
        print('Successifully write: ', slic)

coronacases_org_001.nii 301
Successifully write:  0
Successifully write:  1
Successifully write:  2
Successifully write:  3
Successifully write:  4
Successifully write:  5
Successifully write:  6
Successifully write:  7
Successifully write:  8
Successifully write:  9
Successifully write:  10
Successifully write:  11
Successifully write:  12
Successifully write:  13
Successifully write:  14
Successifully write:  15
Successifully write:  16
Successifully write:  17
Successifully write:  18
Successifully write:  19
Successifully write:  20
Successifully write:  21
Successifully write:  22
Successifully write:  23
Successifully write:  24
Successifully write:  25
Successifully write:  26
Successifully write:  27
Successifully write:  28
Successifully write:  29
Successifully write:  30
Successifully write:  31
Successifully write:  32
Successifully write:  33
Successifully write:  34
Successifully write:  35
Successifully write:  36
Successifully write:  37
Successifully write:  38
Success

Successifully write:  32
Successifully write:  33
Successifully write:  34
Successifully write:  35
Successifully write:  36
Successifully write:  37
Successifully write:  38
Successifully write:  39
Successifully write:  40
Successifully write:  41
Successifully write:  42
Successifully write:  43
Successifully write:  44
Successifully write:  45
Successifully write:  46
Successifully write:  47
Successifully write:  48
Successifully write:  49
Successifully write:  50
Successifully write:  51
Successifully write:  52
Successifully write:  53
Successifully write:  54
Successifully write:  55
Successifully write:  56
Successifully write:  57
Successifully write:  58
Successifully write:  59
Successifully write:  60
Successifully write:  61
Successifully write:  62
Successifully write:  63
Successifully write:  64
Successifully write:  65
Successifully write:  66
Successifully write:  67
Successifully write:  68
Successifully write:  69
Successifully write:  70
Successifully write:  71


IndexError: index 200 is outside the extent for dimension 2 with size 200

In [34]:
help(sitk.GetArrayViewFromImage)

Help on function GetArrayViewFromImage in module SimpleITK.extra:

GetArrayViewFromImage(image)
    Get a NumPy ndarray view of a SimpleITK Image.
    
    Returns a Numpy ndarray object as a "view" of the SimpleITK's Image buffer. This reduces pixel buffer copies, but
     requires that the SimpleITK image object is kept around while the buffer is being used.



In [33]:
help(writer.Execute)

Help on method Execute in module SimpleITK.SimpleITK:

Execute(*args) method of SimpleITK.SimpleITK.ImageFileWriter instance
    Execute(ImageFileWriter self, Image arg2)
    Execute(ImageFileWriter self, Image arg2, std::string const & inFileName, bool useCompression, int compressionLevel)



In [25]:
writer.SetFileName(os.path.join("D:\Studia\sem7\inzynierka\data\eksperyment",'halo.dcm'))
writer.Execute(imgFiltered)

RuntimeError: Exception thrown in SimpleITK ImageFileWriter_Execute: D:\a\1\sitk-build\ITK\Modules\IO\GDCM\src\itkGDCMImageIO.cxx:1328:
itk::ERROR: itk::ERROR: GDCMImageIO(0000021D3DC59F50): DICOM does not support this component type

In [ ]:
help(image3D)

In [15]:
from __future__ import print_function

import SimpleITK as sitk

import sys, time, os
import numpy as np

# if len( sys.argv ) < 2:
#     print( "Usage: python " + __file__ + "<output_directory>" )
#     sys.exit ( 1 )

def writeSlices(series_tag_values, new_img, i):
    image_slice = new_img[:,:,i]

    # Tags shared by the series.
    list(map(lambda tag_value: image_slice.SetMetaData(tag_value[0], tag_value[1]), series_tag_values))

    # Slice specific tags.
    image_slice.SetMetaData("0008|0012", time.strftime("%Y%m%d")) # Instance Creation Date
    image_slice.SetMetaData("0008|0013", time.strftime("%H%M%S")) # Instance Creation Time

    # Setting the type to CT preserves the slice location.
    image_slice.SetMetaData("0008|0060", "CT")  # set the type to CT so the thickness is carried over

    # (0020, 0032) image position patient determines the 3D spacing between slices.
    image_slice.SetMetaData("0020|0032", '\\'.join(map(str,new_img.TransformIndexToPhysicalPoint((0,0,i))))) # Image Position (Patient)
    image_slice.SetMetaData("0020,0013", str(i)) # Instance Number
    
    castFilter = sitk.CastImageFilter()
    castFilter.SetOutputPixelType(sitk.sitkInt16)

    # Convert floating type image (imgSmooth) to int type (imgFiltered)
    imgFiltered = castFilter.Execute(image_slice)

    #sitk.WriteImage(imgFiltered, 'denoised.dcm')

    # Write to the output directory and add the extension dcm, to force writing in DICOM format.
    writer.SetFileName(os.path.join("D:\Studia\sem7\inzynierka\data\eksperyment",str(i)+'.dcm'))
    writer.Execute(imgFiltered)



In [7]:
# Create a new series from a numpy array
# new_arr = np.random.uniform(-10, 10, size = (3,4,5)).astype(np.int16)
new_arr = CT_array
new_img = sitk.GetImageFromArray(new_arr)
new_img.SetSpacing([2.5,3.5,4.5])

In [8]:
# Write the 3D image as a series
# IMPORTANT: There are many DICOM tags that need to be updated when you modify an
#            original image. This is a delicate opration and requires knowlege of
#            the DICOM standard. This example only modifies some. For a more complete
#            list of tags that need to be modified see:
#                           http://gdcm.sourceforge.net/wiki/index.php/Writing_DICOM
#            If it is critical for your work to generate valid DICOM files,
#            It is recommended to use David Clunie's Dicom3tools to validate the files 
#                           (http://www.dclunie.com/dicom3tools.html).

writer = sitk.ImageFileWriter()
# Use the study/series/frame of reference information given in the meta-data
# dictionary and not the automatically generated information from the file IO
writer.KeepOriginalImageUIDOn()

In [9]:
modification_time = time.strftime("%H%M%S")
modification_date = time.strftime("%Y%m%d")

# Copy some of the tags and add the relevant tags indicating the change.
# For the series instance UID (0020|000e), each of the components is a number, cannot start
# with zero, and separated by a '.' We create a unique series ID using the date and time.
# tags of interest:
direction = new_img.GetDirection()
series_tag_values = [("0008|0031",modification_time), # Series Time
                  ("0008|0021",modification_date), # Series Date
                  ("0008|0008","DERIVED\\SECONDARY"), # Image Type
                  ("0020|000e", "1.2.826.0.1.3680043.2.1125."+modification_date+".1"+modification_time), # Series Instance UID
                  ("0020|0037", '\\'.join(map(str, (direction[0], direction[3], direction[6],# Image Orientation (Patient)
                                                    direction[1],direction[4],direction[7])))),
                  ("0008|103e", "Created-SimpleITK")] # Series Description

In [16]:
# Write slices to output directory
list(map(lambda i: writeSlices(series_tag_values, new_img, i), range(new_img.GetDepth())))

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [18]:
# Re-read the series
# Read the original series. First obtain the series file names using the
# image series reader.
# data_directory = sys.argv[1]
data_directory = r"D:\Studia\sem7\inzynierka\data\eksperyment"
series_IDs = sitk.ImageSeriesReader.GetGDCMSeriesIDs(data_directory)
if not series_IDs:
    print("ERROR: given directory \""+data_directory+"\" does not contain a DICOM series.")
    sys.exit(1)
series_file_names = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(data_directory, series_IDs[0])

series_reader = sitk.ImageSeriesReader()
series_reader.SetFileNames(series_file_names)

# Configure the reader to load all of the DICOM tags (public+private):
# By default tags are not loaded (saves time).
# By default if tags are loaded, the private tags are not loaded.
# We explicitly configure the reader to load tags, including the
# private ones.
series_reader.LoadPrivateTagsOn()
image3D = series_reader.Execute()
print(image3D.GetSpacing(),'vs',new_img.GetSpacing())

(2.5, 3.5, 1.0) vs (2.5, 3.5, 4.5)
